In [ ]:
import os
import cv2
from matplotlib import pyplot as plt
import mediapipe as mp
import numpy as np
from math import *

In [ ]:
from sklearn.model_selection import train_test_split # Scikit-learn for our evaluation metrics
from keras.utils.np_utils import to_categorical

# 02 MEDIAPIPE 

In [ ]:
mp_face_mesh = mp.solutions.face_mesh              # FaceMesh Model has -- 478 -- Keypoints
mp_drawing_styles = mp.solutions.drawing_styles    # Drawing styles
mp_drawing = mp.solutions.drawing_utils

### Given the original image and obtaining the values through the model

In [ ]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)   # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                     # Image is no longer writeable
    results = model.process(image)                    # Make prediction
    image.flags.writeable = True                      # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)    # COLOR CONVERSION RGB 2 BGR
    return image, results

### Storing the keypoints values in the mediapipe into a variables

In [ ]:
# face bounder indices
FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]

# Left eyes indices
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]

### Calculating blink rate (Euclidian distance)

In [ ]:
# Blinking Ratio
def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eyes
    # horizontal line
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]
    # draw lines on right eyes
    # cv.line(img, rh_right, rh_left, utils.GREEN, 2)
    # cv.line(img, rv_top, rv_bottom, utils.WHITE, 2)

    # LEFT_EYE
    # horizontal line
    lh_right = landmarks[left_indices[0]]
    lh_left = landmarks[left_indices[8]]

    # vertical line
    lv_top = landmarks[left_indices[12]]
    lv_bottom = landmarks[left_indices[4]]

    rhDistance = euclaideanDistance(rh_right, rh_left)
    rvDistance = euclaideanDistance(rv_top, rv_bottom)

    lvDistance = euclaideanDistance(lv_top, lv_bottom)
    lhDistance = euclaideanDistance(lh_right, lh_left)

    reRatio = rhDistance/rvDistance
    leRatio = lhDistance/lvDistance

    ratio = (reRatio+leRatio)/2
    return ratio

# Euclaidean distance
def euclaideanDistance(point, point1):
    x, y = point
    x1, y1 = point1
    distance = sqrt((x1 - x)**2 + (y1 - y)**2)
    if 1> distance:
        return 1
    else:
        return distance

# 03 EXTRACT POSITION LENDMARKS (KEYPOINTS VALUES)

In [ ]:
def extract_landmarks(img, results):
    frame = cv2.resize(image, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    mesh_coords = mesh_coord_detection(img, results)

    # extract face 468 keyponits
    face = np.array([[res.x, res.y, res.z] for res in results.multi_face_landmarks[0].landmark]).flatten() if results.multi_face_landmarks[0] else np.zeros(478*3)

    # extract right-eye 16 keypoints
    right_eye_coords = np.array([mesh_coords[p] for p in RIGHT_EYE]).flatten()

    # extract left-eye 16 keypoints
    left_eye_coords = np.array([mesh_coords[p] for p in LEFT_EYE]).flatten()

    # calculating eye ratio
    ratio = np.array(blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)).flatten()

    # return all feature values
    #return np.concatenate([face,right_eye_coords,left_eye_coords,ratio])
    return np.concatenate([face, ratio])


def mesh_coord_detection(img, results):
    if img is None:
        img_height, img_width= 0,0
    else:
        img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark] if results.multi_face_landmarks[0] else np.zeros(478 *2)

    # returning the list of tuples for each landmarks
    return mesh_coord

# 04 SETUP FOLDER FOR COLLECTION 

In [ ]:
DATA_PATH = r'D:\Курсовая\truelie\ML_Data'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)        # Path for exported data, numpy arrays
# C:\Users\Sms Studio\Desktop\University_Research_Projects\LieDetection\Semester Research Program\ML_Data
actions = np.array(['true', 'lie'])       # Actions that we try to detect

true_vdos = 156                       # for taking total true videos. 30,30 , 63,63, 50,25 , 25,50, 100,50 , 156,64

lie_vdos = 64                  # for taking total lie videos.

sequence_length = 30             # Videos are going to be 30 frames in length

In [ ]:
# Create Folders for save keypoints values

for action in actions:
    if action == 'true':
        for sequence in range(true_vdos):
            try:
                os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            except:
                pass
    else:
        for sequence in range(lie_vdos):
            try:
                os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            except:
                pass

# 05 COLLECT KEYPOINTS VALUES FOR TRAINING AND TESTING

In [ ]:
for action in actions:   # Loop through actions
    if action == 'true':
        for sequence in range(true_vdos):          # Loop through sequences (true videos)
            VDO_PATH = 'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4'  # /Datasets/true/0/0.mp4
            cap = cv2.VideoCapture(VDO_PATH)                # Read Video
            with mp.solutions.face_mesh.FaceMesh(                         # set mediapipe model
                max_num_faces =1,
                refine_landmarks = True,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as face_mesh:
                for frame_num in range(sequence_length):

                    ret, frame = cap.read() # return status(boolean) , image                               # Read feed

                    print("Processing Video Path:",VDO_PATH)                      # Processing Video Path:Datasets/true/0/0.mp4

                    image, results = mediapipe_detection(frame, face_mesh)                # Make detection

                    feature_values = extract_landmarks(image,results)                   # Extract face , ratio

                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) # create export keypoints local path


                # Keypoints Shape:(1435,) ---> (1434 + 1)

#                   pd.DataFrame(face).to_csv(path)

                    np.save(npy_path, feature_values)                                   # Save Keypoints values

                cap.release()     # Release feed

                cv2.waitKey(1)    # wait for 1 Secound

                print("_____Finish_______")
    else:
        for sequence in range(lie_vdos):          # Loop through sequences (true videos)
            VDO_PATH = 'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4'  # /Datasets/lie/0/0.mp4
            cap = cv2.VideoCapture(VDO_PATH)                # Read Video
            with mp.solutions.face_mesh.FaceMesh(                         # set mediapipe model
                max_num_faces =1,
                refine_landmarks = True,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5) as face_mesh:
                for frame_num in range(sequence_length):

                    ret, frame = cap.read()                                # Read feed

                    print("Processing Video Path:",VDO_PATH)                      # Processing Video Path:Datasets/lie/0/0.mp4

                    image, results = mediapipe_detection(frame, face_mesh)                # Make detection

                    feature_values = extract_landmarks(image,results)                   # Extract face keypoints

                    npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num)) # create export keypoints local path

                    # Keypoints Shape:(1405,) ---> (1404 + 1)

                    np.save(npy_path, feature_values)                                   # Save Keypoints values



                cap.release()     # Release feed

                cv2.waitKey(1)    # wait for 1 Secound

                print("_____Finish_______")

Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing Video Path: Datasets/true/0/0.mp4
Processing

# PREPROCESS DATA AND CREATE LABLES AND FEATURES

In [ ]:
feature_values.shape

(1435,)

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}  # Actions Label Map  {'true': 0, 'lie': 1}

In [ ]:
sequences, labels = [], []                 # like : Features(X) Data , labels(Y) Data
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []                            # Save all of diff.frames paricular sequence
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))   # load np array
            window.append(res)
            print('Processing Video : '+'Datasets/' + action +'/' + str(sequence) + '/' + str(sequence) + '.mp4')
        sequences.append(window)
        labels.append(label_map[action])

Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video : Datasets/true/0/0.mp4
Processing Video

In [102]:
X = np.array(sequences)  # sequences
y = to_categorical(labels).astype(int)  # labels

# Train & Testing Partition
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

In [103]:
X.shape
#y.shape
#X_train.shape
#X_test.shape
#y_train.shape
#y_test.shape

(220, 30, 1435)

# 07 BUILD AND TRAIN LSTM MODELS

In [104]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [105]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [106]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1435)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [107]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [108]:
model.fit(X_train, y_train, epochs=50, callbacks=[tb_callback])

Epoch 1/50
6/6 [==============================] - 3s 56ms/step - loss: 0.7245 - categorical_accuracy: 0.3807
Epoch 2/50
6/6 [==============================] - 0s 51ms/step - loss: 0.6862 - categorical_accuracy: 0.6818
Epoch 3/50
6/6 [==============================] - 0s 53ms/step - loss: 0.6139 - categorical_accuracy: 0.6875
Epoch 4/50
6/6 [==============================] - 0s 55ms/step - loss: 0.6174 - categorical_accuracy: 0.6989
Epoch 5/50
6/6 [==============================] - 0s 53ms/step - loss: 0.6855 - categorical_accuracy: 0.6932
Epoch 6/50
6/6 [==============================] - 0s 53ms/step - loss: 0.6050 - categorical_accuracy: 0.6989
Epoch 7/50
6/6 [==============================] - 0s 55ms/step - loss: 0.5797 - categorical_accuracy: 0.6989
Epoch 8/50
6/6 [==============================] - 0s 61ms/step - loss: 0.5752 - categorical_accuracy: 0.6989
Epoch 9/50
6/6 [==============================] - 0s 78ms/step - loss: 0.5693 - categorical_accuracy: 0.6989
Epoch 10/50
6/6 [==

In [109]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            384000    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 64)                49408     
                                                                 
 dense_9 (Dense)             (None, 64)                4160      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 2)                 66        
                                                                 
Total params: 538,530
Trainable params: 538,530
Non-tr

# MAKE PREDICTIONS

In [110]:
res = model.predict(X_test)

2/2 [==============================] - 0s 10ms/step


In [111]:
X_test.shape

(44, 30, 1435)

In [112]:
np.sum(res[0])

1.0

In [113]:
actions[np.argmax(res[2])]

'true'

In [114]:
y_test

array([[0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0]])

In [115]:
actions[np.argmax(y_test[3])]

'lie'

# SAVE WEIGHTS

In [118]:
model.save('action2.h5')

# Evaluation using Confusion Matrix and Accuracy

In [119]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [120]:
yhat = model.predict(X_test)

2/2 [==============================] - 0s 10ms/step


In [121]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [122]:
ytrue

[1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [123]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[ 0, 11],
        [ 0, 33]],

       [[33,  0],
        [11,  0]]], dtype=int64)

In [124]:
accuracy_score(ytrue, yhat)

0.75

# Test in Real Time

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num].capitalize() + ' ' + str(int(prob*100)) + ' %', (3, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return output_frame

In [53]:
sequence = []
threshold = 0.5

cap = cv2.VideoCapture('riyasTrueVideo.mp4')
# Set mediapipe model
with mp.solutions.face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, face_mesh)
        print(results)

        # 2. Prediction logic
        keypoints = extract_landmarks(image , results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            print(np.array(sequence).shape)
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])

            image = prob_viz(res, actions, image, colors)
        
        # Show to screen
        cv2.imshow('Lie Detection', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

KeyboardInterrupt: 

In [54]:
cap.release()
cv2.destroyAllWindows()